In [11]:
#Capturing frame and landmarks from data
import cv2
import mediapipe as mp
import csv

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Open your training video
cap = cv2.VideoCapture('run3.avi')

# Create a CSV file to store landmarks
with open('pose_landmarks.csv', mode='a', newline='') as f:
    csv_writer = csv.writer(f)
    
    # Define CSV header (add labels later)
    headers = ['frame', 'label']
    for i in range(33):  # MediaPipe Pose has 33 landmarks
        headers += [f'x_{i}', f'y_{i}', f'z_{i}', f'visibility_{i}']
    csv_writer.writerow(headers)

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process the frame to get pose landmarks
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            # Prepare row data for CSV
            row = [frame_count, 'label_here']  # Use a placeholder label initially
            
            for landmark in results.pose_landmarks.landmark:
                row.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            
            # Write the row to CSV
            csv_writer.writerow(row)
        
        frame_count += 1
    print(frame_count)
    cap.release()
    pose.close()

214


In [12]:
#Normalizing the data
import pandas as pd

# Load the CSV with pose landmarks and labels
df = pd.read_csv('pose_landmarks.csv')

# Normalize landmarks by the position of the left hip (landmark 23)
for i in range(33):
    df[f'x_{i}'] -= df['x_23']  # Normalizing by left hip x-coordinate
    df[f'y_{i}'] -= df['y_23']  # Normalizing by left hip y-coordinate

# Save the normalized data
df.to_csv('pose_landmarks_normalized.csv', index=False)

In [13]:
#Split into Training and Testing Sets
from sklearn.model_selection import train_test_split

# Load the normalized data
df = pd.read_csv('pose_landmarks_normalized.csv')

# Separate features (landmarks) and labels
X = df.drop(columns=['frame', 'label'])
y = df['label']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
#Training a basic classifier using TensorFlow/Keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

# Load your labeled landmark data
# X: the input features (pose landmarks), y: the pose labels
# Assuming data is already in numpy format

X = np.array(X, dtype='float32')  # Ensure the data is in numpy format and type float32
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define a simple neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(set(y)), activation='softmax')  # Output layer
])
'''# Define a neural network model with the correct input shape 
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Dense(64, activation='relu', input_shape=(99,)), 
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(len(set(y)), activation='softmax') # Output layer 
])'''

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

model_path = r'C:\Users\hsekh\AppData\Local\Programs\Microsoft VS Code\my_model.h5' 
model.save(model_path) 
print(f"Model saved to {model_path}")

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

Epoch 1/20


C:\Users\hsekh\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6194 - loss: 0.7911 - val_accuracy: 0.8888 - val_loss: 0.3023
Epoch 2/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9126 - loss: 0.2603 - val_accuracy: 0.9133 - val_loss: 0.2221
Epoch 3/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9294 - loss: 0.1983 - val_accuracy: 0.9557 - val_loss: 0.1444
Epoch 4/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9485 - loss: 0.1429 - val_accuracy: 0.9529 - val_loss: 0.1318
Epoch 5/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9600 - loss: 0.1215 - val_accuracy: 0.9698 - val_loss: 0.0934
Epoch 6/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9727 - loss: 0.0901 - val_accuracy: 0.9727 - val_loss: 0.0777
Epoch 7/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9689 - loss: 0.0848 - val_accuracy: 0.9670 - val_loss: 0.0870
Epoch 8/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9759 - loss: 0.0738 - val_accuracy: 0.9793 - val_

Model saved to C:\Users\hsekh\AppData\Local\Programs\Microsoft VS Code\my_model.h5
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9925 - loss: 0.0204
Test accuracy: 0.9915174245834351


In [17]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model(r'C:\Users\hsekh\AppData\Local\Programs\Microsoft VS Code\my_model.h5')  # Make sure to provide the correct path to your model

# Define label map (update with your specific labels)
label_map = {0: "push", 1: "kick", 2: "run"}  # Adjust based on your model's output labels

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Open the video capture
video_path = r'C:\Users\hsekh\AppData\Local\Programs\Microsoft VS Code\run4.avi'
cap = cv2.VideoCapture(video_path)

# Use Pose with the proper confidence parameters
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or no frame to capture.")
            break
        
        # Convert BGR image to RGB for MediaPipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image and detect pose landmarks
        results = pose.process(image)

        # Draw landmarks on the original BGR image
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            # Draw landmarks
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )
            
            # Prepare landmarks for the model
            landmarks = []
            for lm in results.pose_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])  # Append each landmark as x, y, z, visibility

            # Convert landmarks to numpy array and reshape for the model
            landmarks = np.array(landmarks).reshape(1, -1)  # Shape (1, N) where N is the total landmarks data
            
            # Predict the pose
            prediction = model.predict(landmarks)
            predicted_label = np.argmax(prediction)  # Get the index of the highest probability
            label_text = label_map[predicted_label]  # Map the index to a human-readable label
            
            # Display the label on the video
            cv2.putText(image, f"Pose: {label_text}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (255, 255, 255), 2, cv2.LINE_AA)

        # Display the image with the pose label
        cv2.imshow("Pose Detection", image)

        # Exit with 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━